In [65]:
#匯入套件
import re, os, time
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait          # 面對動態網頁,等待元素出現的工具,通常跟EC搭配
from selenium.webdriver.support import expected_conditions as EC # 搭配WebDriverWait,對元素的期待條件,若條件發生,則等待結束,往下一行執行
from selenium.webdriver.common.by import By                      # 期待元素出現要透過什麼方式指定,通常搭配EC,WebDriverWait一起使用  
from fake_useragent import UserAgent
ua = UserAgent(cache=True)

In [67]:
#設定瀏覽器
my_options=webdriver.ChromeOptions()
my_options.add_argument("--start-maximized")
my_options.add_argument("--disanle-popup-blocking")
my_options.add_argument("--disable-notifications")
my_options.add_argument("--lang=zh-TW")

my_service=Service(executable_path="./chromedriver.exe")
driver=webdriver.Chrome(
    options= my_options,
    service= my_service
)
url='https://www.gutenberg.org/browse/languages/zh'
my_headers = {
    'user-agent': ua.random
}
time01=time.time()
driver.get(url)

try:
    WebDriverWait(driver,10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR,"li.pgdbetext>a")))
    
except TimeoutException:
    print('等待逾時!')

listData=[]

folderPath = 'gutenberg_selenium'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

try:
    main_Link=driver.find_elements(By.CSS_SELECTOR,"li.pgdbetext>a")
    for book in main_Link:
        regex01=(r"\b[\u2E80-\u9FFF]+") 
        ch_book=re.match(regex01,book.text)
        if ch_book!=None:
            listData.append(book.text)
    # print(listData)
except NoSuchElementException:
    print("找不到元素!")
    

for index,i in enumerate(range(len(listData))):
    if index>300:
        break
    try:
        driver.find_element(By.LINK_TEXT,listData[i]).click()

    except NoSuchElementException:
        print("找不到元素!")
    
    # sleep(2)
    
    try:
        driver.find_element(By.LINK_TEXT,"Plain Text UTF-8").click()

    except NoSuchElementException:
        print("找不到元素!")
    
    # sleep(2)
    content=driver.find_element(By.CSS_SELECTOR,"pre")
    regex02=r"[\u2E80-\u9FFF]"
    ch_content=re.findall(regex02,content.text)
    ch_content="".join(ch_content)
    Book=listData[i]
    Book=re.sub(r"\s|\:","_",Book)
    if ch_content!=None:
        with open(f'{folderPath}/{Book}.txt','w',encoding='utf-8') as file:
            file.write(ch_content)
    # sleep(2)
    driver.get(url)
driver.quit()
print(f"總執行時間:{time.time()-time01}")

總執行時間:539.9809744358063
